# Good Eggs Item Sales Forecast Model

## Imports

In [ ]:
import calendar
from covid19dh import covid19
import dagstermill as dm
from dagster import ExpectationResult, EventMetadataEntry
import datarobot as dr
from datarobot import Project, Deployment
import datetime
import itertools
import json
import math
import matplotlib
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import requests
import sched, time
import seaborn as sns
import snowflake.connector
import statsmodels as sm
import urllib.request
import warnings

## Settings

In [ ]:
sns.set(style="darkgrid")

## DataRobot API

In [ ]:
DATAROBOT_ENDPOINT = os.environ["DATAROBOT_ENDPOINT"]
DATAROBOT_API_TOKEN = os.environ["DATAROBOT_API_TOKEN"]

try:
    dr.Client(token=DATAROBOT_API_TOKEN,
              endpoint=DATAROBOT_ENDPOINT)
    dr_available = True
except:
    dr_available = False

dr_available

### Manage projects

In [ ]:
try:
    project_list = dr.Project.list(search_params={'project_name': 'item_forecast'})
except:
    project_list = None
project_list

In [ ]:
try:
    project_names = [x.project_name for x in project_list]
except:
    project_names = []
project_names

In [ ]:
num_retained_projects = 5
num_retained_projects

In [ ]:
projects_sorted = sorted(project_names, reverse=True)
projects_sorted

In [ ]:
try:
    project_most_recent = dr.Project(projects_sorted[0])
except:
    project_most_recent = None
project_most_recent

In [ ]:
projects_to_delete = projects_sorted[num_retained_projects:]
projects_to_delete

## Get initial dataset from Snowflake

In [ ]:
SNOWFLAKE_USER = os.environ["SNOWFLAKE_USER"]
SNOWFLAKE_PASSWORD = os.environ["SNOWFLAKE_PASSWORD"]
SNOWFLAKE_ACCOUNT = os.environ["SNOWFLAKE_ACCOUNT"]


with snowflake.connector.connect(user=SNOWFLAKE_USER, password=SNOWFLAKE_PASSWORD, account=SNOWFLAKE_ACCOUNT) as conn:
    items = conn.cursor().execute("SELECT * FROM models.data_science.data_science_items_batch_temperature where calendar_date < current_date()").fetch_pandas_all()
    df_calendar = conn.cursor().execute("SELECT calendar_date, holiday, closed FROM models.goodeggs.calendar").fetch_pandas_all()

In [ ]:
items.columns = map(str.lower, items.columns)

In [ ]:
# Clean up holiday and closed data

In [ ]:
items.sort_values(by=['calendar_date', 'batch', 'temperature_zone'], ignore_index=True, inplace=True)

In [ ]:
items.tail(10)

## Augment Sales Data

### Filter out floral and virtual zones.

In [ ]:
items['temperature_zone'].unique()

In [ ]:
exclude_cols = ['floral', 'flowers', 'virtual']
exclude_cols

In [ ]:
items = items[~items.temperature_zone.isin(exclude_cols)]

In [ ]:
items.shape

### Batch and Temperature Zone Parameters

In [ ]:
n_batches = len(items['batch'].unique())
n_batches

In [ ]:
n_temp_zones = len(items['temperature_zone'].unique())
n_temp_zones

In [ ]:
df_calendar.shape

In [ ]:
df_calendar.columns = map(str.lower, df_calendar.columns)

In [ ]:
df_calendar.columns

In [ ]:
df_calendar.sort_values(by=['calendar_date'], inplace=True)

In [ ]:
df_calendar = df_calendar[['calendar_date', 'holiday', 'closed']]

In [ ]:
df_calendar.tail()

### COVID-19 Cases

https://covid19datahub.io/articles/api/python.html

In [ ]:
df_covid, _ = covid19("USA", level = 2, start = datetime.date(2020, 1, 1), verbose=False)

In [ ]:
df_covid = df_covid[df_covid['administrative_area_level_2'] == 'California']

In [ ]:
df_covid.tail()

In [ ]:
df_cases = df_covid[['date', 'confirmed']]

In [ ]:
df_cases['count'] = df_cases['confirmed'].diff()

In [ ]:
df_cases['date'] = df_cases['date'].apply(lambda x: x.strftime('%Y-%m-%d'))

### Join sales, calendar, and COVID-19 cases

In [ ]:
items['date'] = items['calendar_date'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [ ]:
items.drop(columns=['calendar_date'], inplace=True)

In [ ]:
items.tail()

In [ ]:
df_calendar['date'] = df_calendar['calendar_date'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [ ]:
df_calendar.drop(columns=['calendar_date'], inplace=True)

In [ ]:
df_sc = pd.merge(items, df_calendar, left_on='date', right_on='date', how='left')

In [ ]:
df_sc.tail()

In [ ]:
df_train = pd.merge(df_sc, df_cases, left_on='date', right_on='date', how='left')

In [ ]:
df_train.drop(columns=['date', 'batch', 'confirmed'], inplace=True)

In [ ]:
df_train['count'].fillna(0, inplace=True)

In [ ]:
df_train.tail()

## DataRobot Model

### Feature and Calendar Settings

In [ ]:
holdout_start_date = None
holdout_duration = None

In [ ]:
advance_vars = ['holiday', 'closed']
feature_settings = []
for av in advance_vars:
    feature_settings.append(dr.FeatureSettings(av,
                                               known_in_advance=True,
                                               do_not_derive=False))
feature_settings

In [ ]:
# country_code = dr.CalendarFile.get_allowed_country_codes()[0]['code']
calendar = dr.CalendarFile.create_calendar_from_country_code('US', "2017-01-01", "2021-12-31")

In [ ]:
calendar

### Set Time Partition

In [ ]:
hours_per_day = 24

In [ ]:
derivation_days = 14
derivation_window = derivation_days * hours_per_day
derivation_window

In [ ]:
forecast_days = 21
forecast_window = forecast_days * hours_per_day
forecast_window

In [ ]:
time_partition = dr.DatetimePartitioningSpecification(
    use_time_series                 = True,
    datetime_partition_column       = 'datetime',
    multiseries_id_columns          = ['temperature_zone'],
    feature_derivation_window_start = -derivation_window,
    feature_derivation_window_end   = 0,
    forecast_window_start           = 8,
    forecast_window_end             = forecast_window,
    feature_settings                = feature_settings,
#     calendar_id                     = calendar.id
)
time_partition

### Create Project

Every week, we create a new project because of the updated dataset (the new week of data for augmentation. DataRobot retrains on the updated dataset and returns the best model according to the leaderboard. We then make a 21-day forecast and store the results.

In [ ]:
date_today = datetime.date.today().strftime('%Y%m%d')
date_today

In [ ]:
run_datetime = datetime.datetime.now().strftime('%Y%m%d%H%M')
run_datetime

In [ ]:
project_name = '_'.join(['item_forecast', run_datetime])
project_name

In [ ]:
project = None

if dr_available:
    try:
        project = dr.Project.create(
            project_name = project_name, 
            sourcedata   = df_train
        )
        project_created = True
    except:
        project_created = False

project_created, project

### Start AutoPilot

In [ ]:
%time

if project_created:

    project.set_target(
            target              = 'qty_sold',      
            mode                = dr.AUTOPILOT_MODE.QUICK , # dr.AUTOPILOT_MODE.FULL_AUTO,
            partitioning_method = time_partition,
            metric              = 'SMAPE',
            worker_count        = -1                        # use all available workers
            )

In [ ]:
autopilot_done = False

start_time = time.time()
while project_created and not autopilot_done:
    current_time = time.time()
    time.sleep(60.0 - ((current_time - start_time) % 60.0))
    status = project.get_status()
    autopilot_done = status['autopilot_done']
    time_stamp = datetime.datetime.fromtimestamp(current_time).strftime('%Y-%m-%d %H:%M:%S')
    # print(time_stamp, status)

# autopilot_done

In [ ]:
autopilot_done

### Get Leaderboard

If we successfully created a project with a new model, then select the best current model and get the predictions. If project creation failed, then fall back to the most recent model and make the predictions with that model. In either case, sort the models by best metric, e.g., lowest SMAPE. This code pulls the DR Blueprints with corresponding scores, then stores them in the Pandas dataframe named scores.

In [ ]:
if not project_created and project_most_recent:
    project = project_most_recent
project

In [ ]:
%time

scores = None

models = []
scores = pd.DataFrame()

lb = project.get_datetime_models()
best_models = sorted(
                    [model for model in lb if model.metrics[project.metric]['backtesting']],  
                    key=lambda m: m.metrics[project.metric]['backtesting'],
                    )

for m in best_models:

    backtest_scores = pd.DataFrame(
        [
            {
                'project_name': project.project_name,
                'project_id': project.id,
                'model_id': m.id,
                'model_type': m.model_type,
                'feature_list': m.featurelist_name,
                'optimization_metric': project.metric,
                'scores': m.metrics,
            }
        ]
    )
    scores = scores.append(backtest_scores, sort=False).reset_index(drop=True)  


scores = scores.join(pd.json_normalize(scores["scores"].tolist())).drop(labels=['scores'], axis=1) 

# Drop Empty Columns
scores = scores[scores.columns.drop(list(scores.filter(regex='crossValidation$')))]

# Rename Columns
scores.columns = scores.columns.str.replace(".backtesting", "_all_bt")
scores.columns = scores.columns.str.replace(".holdout", "_holdout")
scores.columns = scores.columns.str.replace(".validation", "_bt_1")
scores.columns = scores.columns.str.replace(' ', '_')

scores = scores[scores.columns.drop(list(scores.filter(regex='_All_BTScores$')))]

# Filter down Accuracy Metrics 
dr_metrics = scores.filter(regex='SMAPE|RMSE').columns.to_list()
dr_project = ['project_name', 'project_id', 'model_id', 'model_type', 'feature_list']
dr_cols = dr_project + dr_metrics
scores = scores[dr_cols]

scores

### Get the Best Model

Get the best model based on the lowest SMAPE. We will make our predictions with this model.

In [ ]:
best_model = None

smape = scores.loc[scores['SMAPE_all_bt'].notnull()]
best_model = pd.DataFrame(smape.loc[smape.SMAPE_all_bt.idxmin()]).transpose()

best_model

## DataRobot Predictions

### Select Model

In [ ]:
%%time

project_id = best_model['project_id'].values[0]
model_id = best_model['model_id'].values[0]
project = dr.Project.get(project_id)
model   = dr.Model.get(project_id, model_id)
    
project, model

### Derivation Dataframe

In [ ]:
n_test_rows = derivation_days * n_batches * n_temp_zones
n_test_rows

In [ ]:
df_train.tail()

In [ ]:
date_yesterday = (datetime.date.today() - datetime.timedelta(days=1)).strftime('%Y-%m-%d')
datetime_cutoff = ' '.join([date_yesterday, '16:00:00'])
datetime_cutoff

In [ ]:
dfp_past = df_train[df_train['datetime'] <= datetime_cutoff].tail(n_test_rows).copy()

In [ ]:
dfp_past.head()

In [ ]:
dfp_past.tail()

### Future Dataframe

In [ ]:
n_future_rows = forecast_days * n_batches * n_temp_zones
n_future_rows

In [ ]:
dfp_future = df_train.tail(n_future_rows).copy()

In [ ]:
dfp_future.loc[:, 'qty_sold'] = None
dfp_future.loc[:, 'count'] = None

In [ ]:
forecast_date_range = pd.date_range(start=date_today, periods=forecast_days * n_batches, freq='8H').repeat(n_temp_zones)

In [ ]:
dfp_future['datetime'] = forecast_date_range

In [ ]:
dfp_future['date'] = dfp_future['datetime'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [ ]:
dfp_future.drop(columns=['holiday', 'closed'], inplace=True)

In [ ]:
dfp_future = pd.merge(dfp_future, df_calendar, left_on='date', right_on='date', how='left')

In [ ]:
dfp_future.drop(columns=['date'], inplace=True)

In [ ]:
dfp_future.head()

In [ ]:
dfp_future.tail()

### Combine derivation and future frames for prediction

In [ ]:
dfp = pd.concat([dfp_past, dfp_future])
dfp.tail()

In [ ]:
dfp_dr = project.upload_dataset(dfp)

In [ ]:
dfp_dr

In [ ]:
pred_job = model.request_predictions(dataset_id = dfp_dr.id)
predictions = pred_job.get_result_when_complete()

In [ ]:
predictions['prediction'] = predictions['prediction'].round().astype('int')

In [ ]:
pred_cols = ['timestamp', 'series_id', 'prediction']
predictions = predictions[pred_cols]

In [ ]:
predictions.rename(columns={"timestamp": "calendar_date", "series_id": "temperature_zone", "prediction": "number_items_prediction"}, inplace=True)

In [ ]:
predictions.head(10)

In [ ]:
predictions.tail(10)

## Store Predictions

The prediction dataframe will have 21 days of data with 4 predictions for each batch (3), so the total number of rows will be 21 x 4 x 3 = 252.

In [ ]:
dm.yield_result(value=predictions, output_name="df")

## End of Notebook